**CNN ARCHITECTURE AND TRAINING**

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset

# Function to filter the dataset
def filter_indices(dataset, classes):
    indices = []
    for i in range(len(dataset)):
        if dataset.targets[i] in classes:
            indices.append(i)
    return indices

# Function to adjust labels
def adjust_labels(sample):
    image, label = sample
    return image, label - 1  # Subtract 1 to make labels start from 0

# Step 3: Load and Preprocess the Data with Filter for Digits 1 to 4
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset_full = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset_full = datasets.MNIST(root='./data', train=False, transform=transform)

# Filter the datasets to only include digits 1-4 (labels 1, 2, 3, 4)
train_indices = filter_indices(train_dataset_full, [1, 2, 3, 4])
test_indices = filter_indices(test_dataset_full, [1, 2, 3, 4])

# Apply label adjustment to the datasets
train_dataset = Subset(train_dataset_full, train_indices)
train_dataset = [(adjust_labels(train_dataset[i])) for i in range(len(train_dataset))]

test_dataset = Subset(test_dataset_full, test_indices)
test_dataset = [(adjust_labels(test_dataset[i])) for i in range(len(test_dataset))]

# Create DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Step 4: Neural Network Model for 4 Output Classes
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 4)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

model = Net()

# Step 5: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Step 6: Train the Model on Filtered Data
for epoch in range(40):  # loop over the dataset multiple times
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Step 7: Evaluate the Model on Filtered Data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print the accuracy
print(f'Accuracy: {100 * correct / total}%')

# Step 8: Save the Model
torch.save(model.state_dict(), 'mnist_model_1_to_4.pth')
print("Model saved as mnist_model_1_to_4.pth")


Accuracy: 99.83169031017071%
Model saved as mnist_model_1_to_4.pth


**INSTALLING ONNX**

In [31]:
pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 81.2 MB/s eta 0:00:00


**CREATING ONNX FILE**

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model = Net()
model.load_state_dict(torch.load('mnist_model_1_to_4.pth', map_location=device))

model.eval()

dummy_input = torch.randn(1, 1, 28, 28).to(device)

torch.onnx.export(model,
                  dummy_input,
                  "mnist_model_1_to_4.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names=['input'],
                  output_names=['output'])

**CHECK IF ONNX FILE IS CREATED**

In [33]:
import onnx

onnx_model = onnx.load("mnist_model_1_to_4.onnx")
onnx.checker.check_model(onnx_model)